<a href="https://colab.research.google.com/github/RepnaiaE/OvertrainingPrediction/blob/main/overtraining_prediction_py.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
def warning_result (file_path):
  import xml.dom.minidom as md
  import pandas as pd
  from pandas import DataFrame
  from numpy import median

  def get_data (file_path):
    from google.colab import drive
    drive.mount('/content/drive')
  
    doc = md.parse(file_path)

    xml_list = []
    expertise = doc.getElementsByTagName("Record")
    for skill in expertise:
      value = (skill.getAttribute("type"),
              skill.getAttribute("unit"),
              skill.getAttribute("startDate"),
              skill.getAttribute("value"))
      xml_list.append(value)
    return xml_list
  xml_list = get_data (file_path)

#Обработка и подготовка данных к работе
  def data_processing (xml_list):
    df = DataFrame(xml_list, columns=['Type', 'Unit', 'Date', 'value']) 
    result_df = df.drop(columns='Unit') 
    df_filter = df ['Type']. isin (['HKQuantityTypeIdentifierRestingHeartRate', 'HKQuantityTypeIdentifierHeartRateVariabilitySDNN']) 
    result_df = result_df [df_filter]
    result_df['value'] = pd.to_numeric(result_df['value'])
    result_df['Date'] = pd.to_datetime(result_df['Date'].str.partition(' ')[0])
    result_df['Type'] = result_df['Type'].str.replace("HKQuantityTypeIdentifierRestingHeartRate","RHR")
    result_df['Type'] = result_df['Type'].str.replace("HKQuantityTypeIdentifierHeartRateVariabilitySDNN","HRV")
    return result_df
  
  result_df = data_processing (xml_list)

  def data_preparation (result_df):
    df_new = DataFrame(columns=['Date', 'RHR', 'HRV']) 
    df_new['Date'] = result_df[result_df ['Type']. isin (['RHR'])]['Date'].unique()
    for i, row in df_new.iterrows():
      df_new.RHR[i] = result_df.value.loc[(result_df.Date == row.Date) & (result_df.Type == "RHR")] .values[0]
      df_new.HRV[i] = result_df.value.loc[(result_df.Date == row.Date) & (result_df.Type == "HRV")].median()
    return df_new

  df_new = data_preparation (result_df)
#Оценка данных
  def data_evaluation (df_new):
    df_rank = get_rank (df_new)
    df_new ['RHR_Rank'] = ""
    df_new ['HRV_Rank'] = ""
    for i, row in df_new.iterrows():
      df_new.RHR_Rank[i] = df_rank.Rank.loc[(df_rank.RHR_value <= row.RHR)].values[0]
      df_new.HRV_Rank[i] = df_rank.Rank.loc[(df_rank.HRV_value >= row.HRV)].values[0]
    return df_new
  
  def get_rank (df_new):
    df_rank = DataFrame(columns=['Rank', 'RHR_value', 'HRV_value','description']) 
    description = ['очень плохо','ниже среднего','норма','хорошо','превосходно']
    df_rank.description = description
    min_RHR = df_new.RHR.min()
    max_RHR = df_new.RHR.max()
    min_HRV = df_new.HRV.min()
    max_HRV = df_new.HRV.max()

    i = [1,2,3,4,5]
    for j in i:
      df_rank.Rank[j-1] = j
      df_rank.RHR_value[j-1] = min_RHR+(5-j)*(max_RHR-min_RHR)/5
      df_rank.HRV_value[j-1] = min_HRV+j*(max_HRV-min_HRV)/5
    return df_rank  

  df_new = data_evaluation (df_new)

 #метод кредитов 
  def pred_score (df_new):
    df_new ['Credit_Energy'] = ""
    df_new ['Pred_Score'] = ""
    df_new.Pred_Score[0] = df_new.RHR_Rank[0]
    df_new.Pred_Score[3] = df_new.RHR_Rank[3]
    for i, row in df_new.iterrows():
      if (i>=3):
        Credit = df_new.RHR_Rank[i]+((df_new.HRV_Rank[i]-df_new.HRV_Rank[i-1])+(df_new.HRV_Rank[i-1]-df_new.HRV_Rank[i-2])+(df_new.HRV_Rank[i-2]-df_new.HRV_Rank[i-3]))
        if (Credit > 5 or Credit < 1):
          if (Credit < 1):
            df_new.Credit_Energy[i] = Credit - 1
            df_new.Pred_Score[i] = 1
          else:
            df_new.Credit_Energy[i] = Credit - 5
            df_new.Pred_Score[i] = 5
        else:
          df_new.Credit_Energy[i] = 0
          df_new.Pred_Score[i] = Credit
      else:
        df_new.Credit_Energy[i] = 0
        df_new.Pred_Score[i] = df_new.RHR_Rank[i]
    return df_new

  def delta_values(df_new):
    df_new ['Delta'] = "" #разница между ожидаемой оценкой реальной оценкой RHR
    df_new ['Flag'] = ""

    for i, row in df_new.iterrows():
      df_new.Delta[i] = df_new.RHR_Rank[i]- df_new.Pred_Score[i]
      df_new.Flag[i] = 0
    return df_new

  def pred_score_with_credit (df_new):
    df_new ['GradeWithCredit'] = "" #"выравненная" оценка состояния уже с учетом кредитов 
    df_new.GradeWithCredit = df_new.Pred_Score
    for i, row in df_new.iterrows():
      if (df_new.Credit_Energy[i]!=0):
        balance = df_new.Credit_Energy[i]
        for j in  range(i+3,len(df_new)):
          if (balance !=0 and df_new.Flag[j] != 1):
            if (df_new.Delta[j]>0 and balance>0):
              if (df_new.Delta[j]>=balance):
                df_new.GradeWithCredit[j] = df_new.GradeWithCredit[j] + balance
                balance = 0
                df_new.Flag[j] = 1
              else:
                df_new.GradeWithCredit[j] = df_new.RHR_Rank[j]
                balance = balance - df_new.Delta[j]
                df_new.Flag[j] = 1
            if (df_new.Delta[j]<0 and balance<0):
              if (df_new.Delta[j]<=balance):
                df_new.GradeWithCredit[j] = df_new.GradeWithCredit[j] + balance
                balance = 0
                df_new.Flag[j] = 1
              else:
                df_new.GradeWithCredit[j] = df_new.RHR_Rank[j]
                balance = balance - df_new.Delta[j]
                df_new.Flag[j] = 1
    return df_new
  def credit_method (df_new):
    df_new = pred_score (df_new)
    df_new = delta_values(df_new)
    df_new = pred_score_with_credit (df_new)
    return df_new
  
  df_new = credit_method (df_new)

#Метод градиентного бустинга
  def boost (result_df, df_new):
    !pip install --upgrade catboost
    import catboost
    from catboost import datasets
    from catboost import Pool, CatBoostClassifier

    df_for_cat = data_preparation (result_df)
    df_for_cat ['Grade'] = df_new.GradeWithCredit
    for i, row in df_for_cat.iterrows():
      df_for_cat.Date[i] = int(df_for_cat.Date[i].timestamp())
      df_for_cat.RHR[i] = int(df_for_cat.RHR[i])
      df_for_cat.HRV[i] = int(df_for_cat.HRV[i])
      df_for_cat.Grade[i] = int (df_for_cat.Grade[i])
    train_data = df_for_cat.iloc[28:160] 
    test_data = df_for_cat.iloc[0:50]

    X_train = train_data.drop(columns = ["Grade"])
    Y_train = train_data.drop(columns = ["Date","RHR","HRV"])
    X_test = test_data.drop(columns = ["Grade"])
    Y_test = test_data.drop(columns = ["Date","RHR","HRV"])
    train_pool = Pool(X_train,Y_train)
    test_pool = Pool(X_test)
    catModel = CatBoostClassifier(iterations=42,learning_rate=1,depth=5,verbose=False)
    catModel.fit(train_pool,verbose=False)
    preds = catModel.predict(test_pool)
    Y_test['Preds'] = preds
    return Y_test

#Результат
  Y_test = boost (result_df, df_new)
  def warning(df_new,Y_test):
    notice = ""
    score = Y_test.Preds.tail(1)[len(Y_test)-1]
    i = len(df_new)-1
    df_res = df_new['GradeWithCredit'].tail(4)
    res = (df_res[i]-df_res[i-1])+(df_res[i-1]-df_res[i-2])+(df_res[i-2]-df_res[i-3])
    if (res > 0):
      notice = "Отлично! Ваше состояние улучшилось!/n Ваша оценка на сегодня = " + str(score)
    else:
      if (res == 0):
        notice = "Ваше состояние осталось в пределах нормы./n Ваша оценка на сегодня = " + str(score)
      else:
        notice = "Есть риск перетренированности! Необходимо сделать паузу и восстановиться./n Ваша оценка на сегодня = " + str(score)
    return  notice

  result = warning(df_new,Y_test)
  return result

In [3]:
file_path = '/content/drive/MyDrive/Диплом/export.xml'
print (warning_result (file_path))


Mounted at /content/drive


<ipython-input-2-584ae8c93344>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.RHR[i] = result_df.value.loc[(result_df.Date == row.Date) & (result_df.Type == "RHR")] .values[0]
<ipython-input-2-584ae8c93344>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.HRV[i] = result_df.value.loc[(result_df.Date == row.Date) & (result_df.Type == "HRV")].median()
<ipython-input-2-584ae8c93344>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 MB 9.7 MB/s eta 0:00:00


<ipython-input-2-584ae8c93344>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.RHR[i] = result_df.value.loc[(result_df.Date == row.Date) & (result_df.Type == "RHR")] .values[0]
<ipython-input-2-584ae8c93344>:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_new.HRV[i] = result_df.value.loc[(result_df.Date == row.Date) & (result_df.Type == "HRV")].median()
<ipython-input-2-584ae8c93344>:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-vers

Есть риск перетренированности! Необходимо сделать паузу и восстановиться./n Ваша оценка на сегодня = 4
